# Training a GCN 

## Envrionment setup

Before you start please run `setup_conda_env.sh` first. You will need to make it executable and run it (check README , same process for clean.sh)



In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import networkx as nx
from scipy.sparse import csr_matrix

This dataset is a citation network. For a given paper, it has information about which papers reference it and which papers it references

In this data, we represent each paper as a node. Each node has an ID and the paper is represented as a bag of words (each column represents the presence or absence of a word in the paper).

Each paper is 1 of 7 classes (https://graphsandnetworks.com/the-cora-dataset/)

**It's not the case that every paper is cited, but every paper cites atleast one other paper in the dataset : the graph is fully connected**

## Load the data

In [36]:
# Load papers
# Here, each  line is a paper
with open('cora/cora.content') as file:
    papers = file.readlines()
# This will be loaded tab seperated, so let's remove the tabs
# and get a list of papers
papers = [p.split('\t') for p in papers]
# The first element is the paper ID and the last is the group classification
# so lets get those
ids = [p[0] for p in papers]
labels = [p[-1] for p in papers]
ids[0], labels[-1]

('31336', 'Neural_Networks\n')

In [ ]:
# let's take a look at the labels
# use set to get unique values
# check the raw strings
for l in set(labels):
    print(repr(l))
# remove the new line characters
labels = [l.strip() for l in labels]
labels[0]

In [ ]:
# Here, each column is the presence or absence of a word in the paper
# the first column is the paper ID and the last column is the label
for i in range(5):
    print(papers[i])

In [ ]:
# Load the citation data
with open('cora/cora.cites') as file:
    cites = file.readlines()

# check the raw strings
# this shows A cites B as A \tab B
print(repr(cites[0]))
#  remove the new line, make the links sublists
cites = [c.strip().split('\t') for c in cites]

In [ ]:
# [i,j] means j cites i. 
# This only has the incoming edges to all the papers
# So here  paper 1033 cites paper 35, etc.
cites[:5]

# Building the graph representations

### The nodes

In [ ]:
# let's make them dataframes for easy manipulation
papers_df = pd.DataFrame(papers)

papers_df[papers_df.columns[0:1433]] = papers_df[papers_df.columns[0:1433]].astype(int)
# Let's call  papers X and remove the ID and label
# Each row represents a node
X = papers_df.drop(columns = [0,1434])      
# do integer encoding of the text labels : map the labels to integers
le = LabelEncoder()
y  = le.fit_transform(labels)
papers_df.head()
# features          
print(X.head())
# labels
print(y[:5])

### The adjacency matrix

The adjacency matrix is a square matrix that tells us how the nodes in a graph are connected

In [ ]:
# How many papers are there ?
N_papers = papers_df.shape[0]
# a dictionary for reindexing 
# Why ? Because these papers are a sample of a bigger dataset, so the paper IDs are not contiguous.
# so we map the paper IDs to contiguous integers
to_new_index = {int(paper_id) : index for index, paper_id in enumerate(papers_df[0])}
index_to_paper = { int(index) : int(paper_id) for paper_id, index in to_new_index.items()}

# map all the cites to the new index
cites_new_index = [[to_new_index[int(cite[0])], to_new_index[int(cite[1])]] for cite in cites]

for c in cites_new_index[:5]:
    print(f"paper {c[1]} --cites--> paper {c[0]}")

In [9]:
# We now create a matrix representation for our adjacency information.
# what we want is something like this:
# where each column represents a citing paper and each row represents a cited paper.
#
#         Citing Paper →
#        0    1    2    3    4
#      +-------------------------+
#  0  |  0    0    0    1    0  |  <-- Paper 0 is cited by Paper 3 
#  1  |  1    0    0    0    0  |  <-- Paper 1 is cited by Paper 0 
#  2  |  0    1    0    0    0  |  <-- Paper 2 is cited by Paper 1 
#  3  |  1    0    1    0    0  |  <-- Paper 3 is cited by Paper 0 and Paper 2 
#  4  |  0    0    0    0    0  |  <-- Paper 4 is not cited by any paper
#      +-------------------------+
#
# In this matrix:
# - The columns represent the citing papers.
# - The rows represent the cited papers.
# - A value of 1 at entry (r, c) indicates that paper c cites paper r.
#

# If we have a very larg network, having a massive matrix with a lot of 0s is unncessary
# https://youtu.be/Qi7FcjN7nsc?si=3_lKyS6MlgD2IWKy
# We make a csr-representation. Instead of holding the full matrix, we maintain  information that
# will defines it.

import scipy.sparse as sp
# turn the cites into a numpy array:
cites_new_index_arr = np.array(cites_new_index)

# cites_new_index_arr looks like:
# [[cited, citer],
#  [cited, citer],
#  ......]
# We get the cited papers : all the values in the first column
row =  cites_new_index_arr[:,0]
# We get the citing papers : all the values in the second column
col = cites_new_index_arr[:,1]
# We want to place a one at each [cited, citer] element
values = np.ones(len(col)) 
# This puts a 1 in every [cited, citer] element, holding only these non-zero elements which conserves space.
A  = sp.csr_matrix((values,(row,col)), shape=(N_papers,N_papers))
A = csr_matrix(A)  

In [ ]:
# It still has the same shape technically, but it is a sparse matrix
print(A.shape)

In [ ]:
# Create a directed graph
G = nx.DiGraph()
import random
# Add edges: the edge is directed from "citing" to "cited"
# lets look at 15 edges
for cited, citing in cites_new_index[200:215]:
    G.add_edge(citing, cited)
# Generate a layout for our nodes (here we use a circular layout)
pos = nx.circular_layout(G)

# Draw nodes, edges, and labels
plt.figure(figsize=(8, 8))
nx.draw_networkx_nodes(G, pos, node_color='skyblue', node_size=700)
nx.draw_networkx_edges(G, pos, arrowstyle='->', arrowsize=20, edge_color='gray')
nx.draw_networkx_labels(G, pos, font_size=12, font_color='black')

plt.title("Citation Network Visualization")
plt.axis('off')
plt.show()

### The node degrees

In [12]:
#  Node degree
# The node degree answers the question : how many connections does a given node have?

# We add self-loops because we will need this for the normalization step coming next
A  += np.diag(np.ones(N_papers))
degrees = []
for i in range(A.shape[0]): # for each paper
    incoming = np.sum(A[i,:])  # Papers that cite paper i (incoming edges)
    outgoing = np.sum(A[:,i])  # Papers that paper i cites (outgoing edges)
    # This counts all connections to a given paper both 'cited by' (rows) and 'is citing' (columns)
    degrees.append(outgoing + incoming)
    if outgoing + incoming < 1.0:
        print(f"Paper {i} ({index_to_paper[i]}) has no connections")

# We create a matrix D which is a diagonal matrix (only the main diagonal has non-zero values).
# This represents the edges that a given paper has
# We could also make it a sparse representation but we skip this for now
D = np.diag(degrees)
adj_matrix = csr_matrix(A)

In [ ]:
print(D)
print(D.shape)

In [ ]:
# lets check
#  paperID of index 0
p = index_to_paper[0]
print(p)
for c in cites:
    if c[0] == str(p) or c[1] == str(p):
        print(c)

#  This is correct. There are 5 edges total but beause we added self loops , we see 7 in the D matrix above.

In [ ]:
# At this point, we have the following important coponents
# X : These are the node features 
#   : This has shape N_papers x BOW features
print(" X shape : ", X.shape)

# A : This is the adacency matrix. It contains the information about which papers are connected to each other
#   : This has shape N_papers x N_papers
print(" A shape : ", adj_matrix.shape)

# D : This is the node degree matrix. It contains the information about how many incoming and outgoing edges each paper has
#   : This has shape N_papers x N_papers
print(" D shape : ", D.shape)

A Graph convolution layer (https://arxiv.org/pdf/1609.02907) applies this function:
$$
H^{(l+1)} = \sigma \left( \tilde{D}^{-\frac{1}{2}} \tilde{A} \tilde{D}^{-\frac{1}{2}} H^{(l)} W^{(l)} \right)
$$

That is, the node features (H) are updated with the connectivity information of each of the nodes in the network (provided by A and D) and W (learnable weights). The learnable weights serve the same role as in a standard neural network where the output of a given layer is  $a = \sigma(WX+b)$.

In [ ]:
 # convert everything to tensors, and rename to match the conventions
X_tensor = torch.Tensor(X.astype(int).to_numpy())
A_tensor = torch.sparse_csr_tensor(
                    torch.from_numpy(adj_matrix.indptr),
                    torch.from_numpy(adj_matrix.indices),
                    torch.from_numpy(adj_matrix.data))
D_tensor = torch.Tensor(D)

# The labels assigned to a given paper from the original dataset
y_tensor= torch.Tensor(y)

# Let'compute the inverse square root of D
# Get the diagonal elements of D
D_diag = torch.diag(D_tensor)
# Compute the inverse square root
D_inv_sqrt = 1 / torch.sqrt(D_diag)
# Create a diagonal matrix from the inverse square root
D_inv_sqrt_diag = torch.diag(D_inv_sqrt)

## Build a GCN

 Let's now build this operation out in PyTorch

In [21]:
# let's define a GCN layer
class GCNlayer(nn.Module):
    def __init__(self, X_in_shape : int, X_out_shape : int,  D_inv_sqrt_diag : torch.Tensor, A : torch.Tensor) -> None:
        super(GCNlayer, self).__init__()
        # Since we want to do X @ W, we need to make sure the dimensions match
        # X has shape N_papers x BOW features
        # W has shape BOW features x W_cols : We can adjust the columns to change the output size
        self.W_rows = X_in_shape
        self.W_cols = X_out_shape
        self.W = nn.Linear(in_features=self.W_rows, out_features=self.W_cols)
        self.D_inv_sqrt_diag = D_inv_sqrt_diag
        self.A = A
    def forward(self, X : torch.Tensor) -> torch.Tensor:
        X_updated = self.W(X) # X @ W
        X_updated = self.D_inv_sqrt_diag @ X_updated # D^(-1/2) @ X @ W
        X_updated = self.A @ X_updated # A @ D^(-1/2) @ X @ W
        # Returns the transformed X of shape N_papers x W_cols
        X_updated = self.D_inv_sqrt_diag @ X_updated
        return X_updated # shape N_papers x W_cols

# We can have any number of layers we want.
class GCN(nn.Module):
    def __init__(self,X_in_shape  : int, layer_out_sizes : list[int], D_inv_sqrt_diag : torch.Tensor, A : torch.Tensor) -> None:
        super(GCN, self).__init__()
        # first layer
        self.layer1 = GCNlayer(X_in_shape= X_in_shape, # = 1433
                               X_out_shape= layer_out_sizes[0], # = 100
                               D_inv_sqrt_diag= D_inv_sqrt_diag,
                               A= A)
        # second layer
        self.layer2 = GCNlayer(X_in_shape= layer_out_sizes[0], # The input shape of the second layer is the output shape of the first layer
                               X_out_shape= layer_out_sizes[1], # = 7
                               D_inv_sqrt_diag= D_inv_sqrt_diag,
                               A= A)
        # softmax layer to get the probability distribution over the classes
        # returns a tensor of shape N_papers x 7
        self.softmax = nn.Softmax(dim= 1)
    def forward(self, X):
        X = torch.relu(self.layer1(X))
        X = self.layer2(X)
        return X

In [ ]:
# Ensure both tensors are of the same type (Float or Double)
X_tensor = X_tensor.to(torch.float32)  # Convert X_tensor to Float
A_tensor = A_tensor.to(torch.float32)  # Convert A_tensor to Float

l = GCNlayer(X_in_shape = X_tensor.shape[1], # This is the number of features in the input node features = BOW features = 1433 
              # This is the number of features in the output node features = 100. We can choose this arbitrarily. 
              # The output of one layer will be of shape X_previous_layer x 100.
             X_out_shape = 100,
             D_inv_sqrt_diag = D_inv_sqrt_diag,
             A = A_tensor)
l.forward(X_tensor).shape



In [ ]:
# test the network
net = GCN(
    X_in_shape=X_tensor.shape[1], # input shape = N_papers x BOW features
    layer_out_sizes=[100, 7], # first layer has 100 features, second layer has 7 features
    D_inv_sqrt_diag=D_inv_sqrt_diag,
    A=A_tensor
)
# We should expect the output to be of shape N_papers x layer_out_sizes after each layer
# and the final output to be of shape N_papers x 7
net(X_tensor).shape


Let's train a network to predict the label for a given node

In [ ]:
# Recall that we 7 classes 
print(set(labels))
# encoded as integers
print(set(y))

In [26]:
# Let's separate the data into training and test sets
# Create boolean masks for training, validation, and testing  with 60% for training, 20% for validation, and 20% for testing
# use train test split to get the indices
from sklearn.model_selection import train_test_split
train_indices, val_test_indices = train_test_split(range(2708), test_size=0.4, random_state=42)
val_indices, test_indices = train_test_split(val_test_indices, test_size=0.5, random_state=42)
# use the indices to get the masks
train_mask = torch.zeros(2708, dtype=torch.bool)
train_mask[train_indices] = True

val_mask = torch.zeros(2708, dtype=torch.bool)
val_mask[val_indices] = True

test_mask = torch.zeros(2708, dtype=torch.bool)
test_mask[test_indices] = True

In [ ]:
# Now we can build and train our network
net = GCN(X_in_shape=X_tensor.shape[1], # input shape = 1433
          layer_out_sizes=[100, 7], # first layer has 100 features, second layer has 7 features
          D_inv_sqrt_diag=D_inv_sqrt_diag,
          A=A_tensor)
net

In [ ]:
# Make sure that only the W matrices are trainable
# Pytorch will automatically set the requires_grad attribute to True for all parameters by default
# Named parameters returns a list of tuples, where each tuple contains the name of the parameter and the parameter itself
# We can use this to check which parameters are trainable
# We should expect only the W matrices to have requires_grad =  True
for name, param in net.named_parameters():
    if param.requires_grad:
        print(name, param.shape)


In [ ]:
# Now we can do a standard training loop
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)
for epoch in range(150):
    net.train()
    optimizer.zero_grad()
    # Each forward pass computes the output for all nodes in the network
    output = net(X_tensor)
    # We only want to compute the loss for the training nodes
    loss = criterion(output[train_mask], y_tensor[train_mask].long())
    loss.backward()
    optimizer.step()

    net.eval()
    with torch.no_grad():
        output = net(X_tensor)
        loss = criterion(output[val_mask], y_tensor[val_mask].long())
        if epoch % 10 == 0:
            val_outputs = net(X_tensor)
            val_accuracy = (val_outputs[val_mask].argmax(1) == y_tensor[val_mask]).float().mean()
            print(f"Validation Accuracy: {val_accuracy.item()}")



In [ ]:
# Test the network
net.eval()
with torch.no_grad():
    output = net(X_tensor)
    test_loss = criterion(output[test_mask], y_tensor[test_mask].long())
    test_accuracy = (output[test_mask].argmax(1) == y_tensor[test_mask]).float().mean()
    print(f"Test Accuracy: {test_accuracy.item()}")